In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import random
import math
import os
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import gudhi as gd
from scipy.ndimage import distance_transform_bf

Raw data (F42B.dat, F42C.dat, LV60A.dat, and LV60C.dat) are downloadable from [The Imperial College Consortium on Pore-Scale Modelling and Imaging website](https://www.imperial.ac.uk/earth-science/research/research-groups/pore-scale-modelling/micro-ct-images-and-networks/)

In [ ]:
# convert list to array
def pdarray(pd):
    pd_array=np.zeros((len(pd),3))
    for i in range(0,len(pd)):
        pd_array[i,0]=np.asarray(pd[i][0])
        pd_array[i,1]=np.asarray(pd[i][1][0])
        pd_array[i,2]=np.asarray(pd[i][1][1])
    return pd_array;

In [ ]:
def computepd(name):
    with open(name+".dat",'r') as f:
    df = pd.DataFrame(l.rstrip().split() for l in f)
    
    dataarray=df.values
    dataarray=dataarray.astype(int)

    data3d=np.zeros((300,300,300))
    for ii in range(1,300):
        data3d[:,:,ii]=dataarray[300*(ii):300*(ii+1),:]
    
    X=100
    Y=100
    Z=100

    for ii in range(0,3):
        for jj in range(0,3):
            for kk in range(0,3):
                a=X*(ii)
                b=X*(ii+1)

                c=Y*(jj)
                d=Y*(jj+1)

                e=Z*(kk)
                f=Z*(kk+1)        

                df_sub=data3d[a:b, c:d, e:f]

                distimgp=distance_transform_bf(df_sub,metric='euclidean')
                distimgn=distance_transform_bf(1-df_sub,metric='euclidean')

                distimgp = distimgp.astype(np.int16)
                distimgn = distimgn.astype(np.int16)

                distimg=distimgp-distimgn+0.00001

                per_disimg=np.ravel(distimg)
                per_disimg_fin=np.array(per_disimg.flatten())

                info=np.array([3,X,Y,Z])

                # write txt file
                outfile = "./data/"+name+"_distance" + str(ii) + str(jj) + str(kk) + ".txt"
                f= open(outfile,"w+")
                for ll in range(0,len(info)):
                    f.write("%d\n" % (info[ll]) )
                for mm in range(0,len(per_disimg_fin)):  
                    f.write("%f\n" % (per_disimg_fin[mm]))
                f.close()

                md_cubical_complex = gd.CubicalComplex(perseus_file= outfile)
                md_cc_diag=md_cubical_complex.persistence()
                pd_array=pdarray(md_cc_diag)
                np.savetxt("./data/"+name+"_pd"+ str(ii) + str(jj) + str(kk) +'.txt',pd_array,fmt='%9f')
                
                os.remove(outfile)

Compute persistence diagrams

In [ ]:
computepd("F42B")
computepd("F42C")
computepd("LV60A")
computepd("LV60C")